In [1]:
import os
import glob
import pandas as pd
import pickle
import numpy as np

from orion.primitives.tadgan import score_anomalies
from orion.primitives.timeseries_anomalies import find_anomalies
from orion.evaluation.contextual import contextual_f1_score, contextual_confusion_matrix
from orion.primitives.timeseries_errors import reconstruction_errors
from orion.primitives.timeseries_errors import regression_errors
from orion.primitives.aer import bi_regression_errors, score_anomalies
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler

In [2]:
RESULTS_DIRECTORY = '/Volumes/easystore/results'
CACHE_DIRECTORY = os.path.join(RESULTS_DIRECTORY, '{pipeline}/cache/**.csv')
TEMP_DIRECTORY = os.path.join(os.getcwd(), 'temp')
F1_DIRECTORY = os.path.join(os.getcwd(), 'f1')

if not os.path.exists(TEMP_DIRECTORY): os.makedirs(TEMP_DIRECTORY)
if not os.path.exists(F1_DIRECTORY): os.makedirs(F1_DIRECTORY)

REGRESSION_PIPELINES = [
    'ARIMA',
    'ARIMA-UCR',
    'U_lstm_dynamic_threshold_2.0',
    'U_lstm_dynamic_threshold_2.0-UCR'
]

BI_REGRESSION_PIPELINES = [
    'U_bi_reg_250_2.0',
    'U_bi_reg_250_2.0-UCR'
]

RECONSTRUCTION_PIPELINES = [
    'U_lstm_autoencoder_2.0',
    'U_lstm_autoencoder_2.0-UCR',
    'U_vae_2.0',
    'U_vae_2.0-UCR'
]

TADGAN_PIPELINES = [
    'U_final_tadgan_2.0_gpu',
    'U_final_tadgan_2.0_gpu-UCR'
]

AER_PIPELINES = [
    'U_bi_reg_ae_expand_weighted_loss_2.0',
    'U_bi_reg_ae_expand_weighted_loss_2.0-UCR'
]


In [3]:
def create_summary(pipeline: str):

    summary_dict = {
        'dataset': [],
        'pipeline': [],
        'signal': [],
        'run_id': [],
        'elapsed': [],
        'expected': [],
        'index': [],
        'y': []
    }


    cache_paths = glob.glob(os.path.join(RESULTS_DIRECTORY, pipeline, 'cache', '*.pkl'))
    for cache_path in cache_paths:
        summary_dict['pipeline'].append(pipeline)

        # Read metadata
        metadata = pd.read_csv(cache_path[:-8] + '_scores.csv').iloc[0].to_dict()
        for col in ['dataset', 'signal', 'expected', 'elapsed', 'run_id']:
            summary_dict[col].append(metadata[col])

        # Read pipeline outputs
        with open(cache_path, 'rb') as f:
            
            if pipeline in REGRESSION_PIPELINES or pipeline in RECONSTRUCTION_PIPELINES:
                _, y, y_hat, index, _ = pickle.load(f)
                summary_dict.setdefault('y_hat', [])
                summary_dict['y_hat'].append(y_hat)
                
            elif pipeline in BI_REGRESSION_PIPELINES:
                if 'UCR' not in pipeline:
                    _, y, fy_hat, ry_hat, index, _, _, _ = pickle.load(f) 
                else:
                    _, y, fry_hat, _, index, _, _, _ = pickle.load(f) 
                    fy_hat, ry_hat = fry_hat
                summary_dict.setdefault('fy_hat', [])
                summary_dict['fy_hat'].append(fy_hat)
                summary_dict.setdefault('ry_hat', [])
                summary_dict['ry_hat'].append(ry_hat)
            
            elif pipeline in TADGAN_PIPELINES:
                _, y, _, _, index, errors, _ = pickle.load(f)  
                summary_dict.setdefault('errors', [])
                summary_dict['errors'].append(errors)
            
            elif pipeline in AER_PIPELINES:
                # Original version viz output is weird.
                _, y, fry_hat, y_hat, index, _, _, _ = pickle.load(f)
                fy_hat, ry_hat = fry_hat
                
                summary_dict.setdefault('y_hat', [])
                summary_dict['y_hat'].append(y_hat)
                summary_dict.setdefault('fy_hat', [])
                summary_dict['fy_hat'].append(fy_hat)
                summary_dict.setdefault('ry_hat', [])
                summary_dict['ry_hat'].append(ry_hat)
                
        summary_dict['index'].append(index)
        summary_dict['y'].append(y)
        

    summary_df = pd.DataFrame(summary_dict)
    return summary_df

In [4]:
def mask_scores(anomaly_scores):
    num_hide_from_start = int(0.01 * len(anomaly_scores))
    anomaly_scores[:num_hide_from_start] = min(anomaly_scores)
    return anomaly_scores

def calculate_regression_scores(y, y_hat):
    anomaly_scores = regression_errors(y, y_hat)
    return anomaly_scores

def calculate_reconstruction_scores(y, y_hat):
    anomaly_scores, _ = reconstruction_errors(y, y_hat, rec_error_type='dtw', score_window=10)
    return anomaly_scores

def calculate_tadgan_scores(scores):
    return scores

def calculate_aer_reconstruction(y, y_hat):
    rec_scores, _ = reconstruction_errors(y[:, 1:-1], y_hat,
                                      smoothing_window=0.01,
                                      smooth=True,
                                      rec_error_type='dtw')
    rec_scores = mask_scores(rec_scores)
    rec_scores = np.concatenate([np.zeros(1), rec_scores, np.zeros(1)])
    return rec_scores

def calculate_scores_wrapper(score_func: callable, params: list, pipeline: str, processes: int = 10):
    """Load cache scores or process scores with multiple workers."""
    
    temp_scores_path = os.path.join(TEMP_DIRECTORY, pipeline + '.pkl')
    
    if os.path.exists(temp_scores_path):
        with open(temp_scores_path, 'rb') as f:
            scores = pickle.load(f)
    
    else:        
        with Pool(processes=processes) as pool:
            scores = pool.starmap(score_func, params)
        
        with open(temp_scores_path, 'wb') as f:
            pickle.dump(scores, f)
    
    return pd.Series(scores)

def calculate_anomaly_scores(summary: pd.DataFrame, pipeline: str, mask: bool = False) -> pd.DataFrame:

    if pipeline in REGRESSION_PIPELINES:
        columns = ['y', 'y_hat']
        score_func = calculate_regression_scores
        built_in_mask = False

    elif pipeline in BI_REGRESSION_PIPELINES:
        columns = ['y', 'ry_hat', 'fy_hat']
        score_func = bi_regression_errors
        built_in_mask = True
        
    elif pipeline in RECONSTRUCTION_PIPELINES:
        columns = ['y', 'y_hat']
        score_func = calculate_reconstruction_scores
        built_in_mask = False
    
    elif pipeline in TADGAN_PIPELINES:
        columns = ['errors']
        score_func = calculate_tadgan_scores
        built_in_mask = False
        
    params = summary[columns].copy()
    params = params.to_records(index=False)
    scores = calculate_scores_wrapper(score_func, params, pipeline)
    
    if not built_in_mask and mask:
        scores = [mask_scores(score) for score in scores]
    summary['scores'] = scores
    return summary

In [5]:
def calculate_summary_from_scores(summary: pd.DataFrame) -> pd.DataFrame:
    summary_dict = {
        'dataset': [],
        'pipeline': [],
        'signal': [],
        'run_id': [],
        'elapsed': [],
        'f1': [],
        'tn': [],
        'fp': [],
        'fn': [],
        'tp': []
    }
    
    for idx, row in summary.iterrows():
            
        # Final anomalies
        anomalies = find_anomalies(row['scores'], 
                                   row['index'],    
                                   window_size_portion=0.33,
                                   window_step_size_portion= 0.1,
                                   fixed_threshold=True,
                                   min_percent=0.13
                                  )
        anomalies = [(int(anomaly[0]), int(anomaly[1])) for anomaly in anomalies]

        # Calculate confusion matrix
        tn, fp, fn, tp = contextual_confusion_matrix(eval(row['expected']), anomalies, weighted=False)
        try:
            f1 = tp / (tp + .5 * (fn + fp))
        except:
            f1 = 0
        
        # Populate dictionary with information
        for i in ['dataset', 'pipeline', 'signal', 'run_id', 'elapsed']:
            summary_dict[i].append(row[i])
        summary_dict['f1'].append(f1)
        summary_dict['tn'].append(tn)
        summary_dict['fp'].append(fp)
        summary_dict['fn'].append(fn)
        summary_dict['tp'].append(tp)
    
    return pd.DataFrame.from_dict(summary_dict)

In [6]:
def load_results_from_output(pipeline: str) -> pd.DataFrame:
    directory = CACHE_DIRECTORY.format(pipeline=pipeline)
    return pd.concat([pd.read_csv(file) for file in glob.glob(directory)])


def calculate_results_for_all_datasets(pipeline: str, mask: bool, direct_output=False) -> pd.DataFrame:
    ucr_pipeline = pipeline + '-UCR'
    
    if direct_output:
        non_ucr_results = load_results_from_output(pipeline)
        ucr_results = load_results_from_output(ucr_pipeline)
    
    else:
        non_ucr_results = calculate_summary_from_scores(calculate_anomaly_scores(create_summary(pipeline), pipeline, mask))
        ucr_results = calculate_summary_from_scores(calculate_anomaly_scores(create_summary(ucr_pipeline), ucr_pipeline, mask))
    
    return pd.concat([non_ucr_results, ucr_results])

In [7]:
def calculate_f1_for_all_datasets(pipeline: str, results: pd.DataFrame, ndigits: int = 4) -> pd.DataFrame:    
    f1_scores = []
    simple_results = {}
    for group, sub_df in results.groupby(['pipeline', 'dataset', 'run_id']):
        metrics = {metric: int(sub_df[metric].sum()) for metric in ['tn', 'fp', 'fn', 'tp']}
        f1 = round(metrics['tp'] / (metrics['tp'] + .5 * (metrics['fn'] + metrics['fp'])), ndigits)
        simple_results[group[1]] = [f1]
        f1_scores.append(f1)
    
    simple_results['Mean'] = round(np.mean(f1_scores), ndigits)
    simple_results['StDev'] = round(np.std(f1_scores), ndigits)
    
    output = pd.DataFrame(simple_results)
    output = output[['MSL', 'SMAP', 'YAHOOA1', 'YAHOOA2', 'YAHOOA3', 'YAHOOA4', 'artificialWithAnomaly', 'realAdExchange', 'realAWSCloudwatch', 'realTraffic', 'realTweets', 'UCR', 'Mean', 'StDev']]
    output.index = [pipeline]
    return output


In [8]:
def save_output(pipeline: str, results: pd.DataFrame, f1: pd.DataFrame) -> None:
    results.to_csv(os.path.join(F1_DIRECTORY, pipeline + '_results.csv'))
    f1.to_csv(os.path.join(F1_DIRECTORY, pipeline + '_f1.csv'))
    
def load_f1(pipeline: str) -> pd.DataFrame:
    return pd.read_csv(os.path.join(F1_DIRECTORY, pipeline + '_f1.csv'), index_col=0)

# Table IV Figure A

## ARIMA

In [34]:
results = calculate_results_for_all_datasets('ARIMA', False, True)
f1 = calculate_f1_for_all_datasets('ARIMA', results)
save_output('ARIMA', results, f1)

## ARIMA (M)*

In [35]:
results = calculate_results_for_all_datasets('ARIMA-M', True, True)
f1 = calculate_f1_for_all_datasets('ARIMA (M)', results)
save_output('ARIMA (M)', results, f1)

## LSTM-NDT

Pipelines: `lstm_dynamic_threshold_viz`

In [25]:
results = calculate_results_for_all_datasets('U_lstm_dynamic_threshold_2.0', False)
f1 = calculate_f1_for_all_datasets('LSTM-NDT', results)
save_output('LSTM-NDT', results, f1)

Proof that current post-processing is the same as output

In [14]:
results = calculate_results_for_all_datasets('U_lstm_dynamic_threshold_2.0', False, True)
f1_direct = calculate_f1_for_all_datasets('LSTM-NDT', results)
assert(f1.equals(f1_direct))

## LSTM-NDT (M) *
Pipelines: `lstm_dynamic_threshold_viz`

In [ ]:
results = calculate_results_for_all_datasets('U_lstm_dynamic_threshold_2.0', True)
f1 = calculate_f1_for_all_datasets('LSTM-NDT (M)', results)
save_output('LSTM-NDT (M)', results, f1)

## LSTM-NDT (M, Bi)*
Pipeline: `bi_reg_250`

In [ ]:
results = calculate_results_for_all_datasets('U_bi_reg_250_2.0', True)
f1 = calculate_f1_for_all_datasets('LSTM-NDT (M, Bi)', results)
save_output('LSTM-NDT (M, Bi)', results, f1)

## LSTM-AE
Pipeline: `lstm_autoencoder_viz` \
Primitive: `keras.Sequential.LSTMSeq2Seq` \
Params: `keras.Sequential.LSTMSeq2Seq`

In [18]:
results = calculate_results_for_all_datasets('U_lstm_autoencoder_2.0', False)
f1 = calculate_f1_for_all_datasets('LSTM-AE', results)
save_output('LSTM-AE', results, f1)

## LSTM-AE (M)*
Pipeline: `lstm_autoencoder_viz` \
Primitive: `keras.Sequential.LSTMSeq2Seq` \
Params: `keras.Sequential.LSTMSeq2Seq`

In [19]:
results = calculate_results_for_all_datasets('U_lstm_autoencoder_2.0', True)
f1 = calculate_f1_for_all_datasets('LSTM-AE (M)', results)
save_output('LSTM-AE (M)', results, f1)

## LSTM-VAE
Pipeline: `vae_viz` \
Primitive: `orion.primitives.vae.VAE` \
Params: `orion.primitives.vae.VAE.json`

In [21]:
results = calculate_results_for_all_datasets('U_vae_2.0', False)
f1 = calculate_f1_for_all_datasets('LSTM-VAE', results)
save_output('LSTM-VAE', results, f1)

## LSTM-VAE (M)*
Pipeline: `vae_viz` \
Primitive: `orion.primitives.vae.VAE` \
Params: `orion.primitives.vae.VAE.json`

In [22]:
results = calculate_results_for_all_datasets('U_vae_2.0', True)
f1 = calculate_f1_for_all_datasets('LSTM-VAE (M)', results)
save_output('LSTM-VAE (M)', results, f1)

## TadGAN
Pipeline: `tadgan_gpu` \
Primitive: `orion.primitives.tadgan.TadGAN` \
Params: `orion.primitives.tadgan.TadGAN.json`

In [10]:
tadgan_results = calculate_results_for_all_datasets('U_final_tadgan_2.0_gpu', False)
tadgan_f1 = calculate_f1_for_all_datasets('TadGAN', tadgan_results)
save_output('TadGAN', tadgan_results, tadgan_f1)

## TadGAN (M)*
Pipeline: `tadgan_gpu` \
Primitive: `orion.primitives.tadgan.TadGAN` \
Params: `orion.primitives.tadgan.TadGAN.json`

In [11]:
tadgan_m_results = calculate_results_for_all_datasets('U_final_tadgan_2.0_gpu', True)
tadgan_m_f1 = calculate_f1_for_all_datasets('TadGAN (M)', tadgan_m_results)
save_output('TadGAN (M)', tadgan_m_results, tadgan_m_f1)

## Complete Table

In [26]:
pipelines = [
    'ARIMA',
    'ARIMA (M)',
    'LSTM-NDT',
    'LSTM-NDT (M)',
    'LSTM-AE',
    'LSTM-AE (M)',
    'LSTM-VAE',
    'LSTM-VAE (M)',
    'TadGAN',
    'TadGAN (M)'
]

pd.concat([load_f1(pipeline) for pipeline in pipelines])

,MSL,SMAP,YAHOOA1,YAHOOA2,YAHOOA3,YAHOOA4,artificialWithAnomaly,realAdExchange,realAWSCloudwatch,realTraffic,realTweets,UCR,Mean,StDev
ARIMA,0.4421,0.3333,0.7332,0.8074,0.8176,0.6998,0.3529,0.7407,0.5176,0.5000,0.5672,0.1236,0.5529,0.2068
ARIMA (M),0.4565,0.3590,0.7519,0.8093,0.8069,0.6899,0.5000,0.7692,0.5176,0.5000,0.5758,0.1481,0.5737,0.1933
LSTM-NDT,0.5155,0.7075,0.7206,0.9801,0.7442,0.6379,0.4000,0.7407,0.5128,0.6667,0.5797,0.3908,0.6330,0.1589
LSTM-NDT (M),0.5208,0.7536,0.7287,0.9875,0.7335,0.6378,0.6000,0.7692,0.5128,0.6857,0.5882,0.4461,0.6637,0.1398
LSTM-AE,0.4571,0.7259,0.6391,0.9610,0.5938,0.3678,0.4444,0.6429,0.6786,0.5926,0.5357,0.3292,0.5807,0.1652
LSTM-AE (M),0.4571,0.7717,0.6448,0.9633,0.5938,0.3670,0.4444,0.6429,0.6786,0.6154,0.5357,0.3333,0.5873,0.1693
LSTM-VAE,0.4324,0.6056,0.6171,0.9124,0.5977,0.3263,0.4444,0.7586,0.7018,0.5926,0.5357,0.3540,0.5732,0.1625
LSTM-VAE (M),0.4384,0.7049,0.6292,0.9406,0.5946,0.3279,0.4444,0.7586,0.7143,0.6154,0.5357,0.3607,0.5887,0.1709
TadGAN,0.5843,0.6174,0.5325,0.8421,0.3907,0.2969,0.5714,0.7200,0.6774,0.5806,0.5882,0.1625,0.5470,0.1777
TadGAN (M),0.5843,0.6301,0.5342,0.8462,0.3950,0.2913,0.6154,0.7200,0.6774,0.5806,0.5882,0.1642,0.5522,0.1796


# Table IV Figure B

Pipeline: `bi_reg` \
Primitive: `bi_reg_ae_expand.py`

In [15]:
pipeline = 'U_bi_reg_ae_expand_weighted_loss_2.0'
summary = create_summary(pipeline)

In [16]:
params = summary[['y', 'ry_hat', 'fy_hat']]
params = params.to_records(index=False)
reg_scores = np.array(calculate_scores_wrapper(bi_regression_errors, params, pipeline + '_bi_reg'))

In [72]:
params = summary[['y', 'y_hat']]
params = params.to_records(index=False)
rec_scores = np.array(calculate_scores_wrapper(calculate_aer_reconstruction, params, pipeline + '_dtw'))

In [9]:
ucr_pipeline = 'U_bi_reg_ae_expand_weighted_loss_2.0-UCR'
ucr_summary = create_summary(ucr_pipeline)

In [12]:
ucr_params = ucr_summary[['y', 'ry_hat', 'fy_hat']]
ucr_params = ucr_params.to_records(index=False)
ucr_reg_scores = calculate_scores_wrapper(bi_regression_errors, ucr_params, ucr_pipeline + '_bi_reg')

In [73]:
ucr_params = ucr_summary[['y', 'y_hat']]
ucr_params = ucr_params.to_records(index=False)
ucr_rec_scores = np.array(calculate_scores_wrapper(calculate_aer_reconstruction, ucr_params, ucr_pipeline + '-UCR_dtw'))

## AER (PRED)*

In [74]:
scores = reg_scores
summary['scores'] = scores
non_ucr_results = calculate_summary_from_scores(summary)

ucr_scores = ucr_reg_scores
ucr_summary['scores'] = ucr_scores
ucr_results = calculate_summary_from_scores(ucr_summary)

results = pd.concat([non_ucr_results, ucr_results])

In [75]:
f1 = calculate_f1_for_all_datasets('AER (PRED)', results)
save_output('AER (PRED)', results, f1)

## AER (SUM)*

In [76]:
def sum_score(reg_scores, rec_scores, lambda_rec = 0.5):
    reg = MinMaxScaler([0, 1]).fit_transform(reg_scores.reshape(-1, 1)).flatten()
    rec = MinMaxScaler([0, 1]).fit_transform(rec_scores.reshape(-1, 1)).flatten()
    scores = (1 - lambda_rec) * reg + lambda_rec * rec
    return scores

scores = [sum_score(reg_score, rec_score) for reg_score, rec_score in zip(reg_scores, rec_scores)]
summary['scores'] = scores
non_ucr_results = calculate_summary_from_scores(summary)


ucr_scores = [sum_score(reg_score, rec_score) for reg_score, rec_score in zip(ucr_reg_scores, ucr_rec_scores)]
ucr_summary['scores'] = ucr_scores
ucr_results = calculate_summary_from_scores(ucr_summary)

results = pd.concat([non_ucr_results, ucr_results])

In [77]:
f1 = calculate_f1_for_all_datasets('AER (SUM)', results)
save_output('AER (SUM)', results, f1)

## AER (REC)*

In [78]:
scores = rec_scores
summary['scores'] = scores
non_ucr_results = calculate_summary_from_scores(summary)

ucr_scores = ucr_rec_scores
ucr_summary['scores'] = ucr_scores
ucr_results = calculate_summary_from_scores(ucr_summary)

results = pd.concat([non_ucr_results, ucr_results])

In [79]:
f1 = calculate_f1_for_all_datasets('AER (REC)', results)
save_output('AER (REC)', results, f1)

## AER (MULT)*

In [80]:
def mult_score(reg_scores, rec_scores, lambda_rec = 0.5):
    reg_scores = MinMaxScaler([1, 2]).fit_transform(reg_scores.reshape(-1, 1)).flatten()
    rec_scores = MinMaxScaler([1, 2]).fit_transform(rec_scores.reshape(-1, 1)).flatten()
    scores = np.multiply(reg_scores, rec_scores)
    return scores

scores = [mult_score(reg_score, rec_score) for reg_score, rec_score in zip(reg_scores, rec_scores)]
summary['scores'] = scores
non_ucr_results = calculate_summary_from_scores(summary)


ucr_scores = [mult_score(reg_score, rec_score) for reg_score, rec_score in zip(ucr_reg_scores, ucr_rec_scores)]
ucr_summary['scores'] = ucr_scores
ucr_results = calculate_summary_from_scores(ucr_summary)

results = pd.concat([non_ucr_results, ucr_results])

In [81]:
f1 = calculate_f1_for_all_datasets('AER (MULT)', results)
save_output('AER (MULT)', results, f1)

In [82]:
pipelines = [
    'AER (PRED)',
    'AER (SUM)',
    'AER (REC)',
    'AER (MULT)'
]

pd.concat([load_f1(pipeline) for pipeline in pipelines])

,MSL,SMAP,YAHOOA1,YAHOOA2,YAHOOA3,YAHOOA4,artificialWithAnomaly,realAdExchange,realAWSCloudwatch,realTraffic,realTweets,UCR,Mean,StDev
AER (PRED),0.5227,0.6944,0.7146,0.9359,0.8815,0.7281,0.8000,0.7097,0.4946,0.7368,0.6000,0.4635,0.6902,0.1413
AER (SUM),0.5060,0.7273,0.7193,0.9381,0.7166,0.5367,0.5000,0.7333,0.7241,0.5882,0.5373,0.4448,0.6393,0.1366
AER (REC),0.4800,0.6992,0.7086,0.9752,0.6207,0.4200,0.4444,0.6429,0.6786,0.5714,0.5000,0.3807,0.5935,0.1582
AER (MULT),0.6111,0.7969,0.7896,0.9586,0.7472,0.5784,0.5000,0.6207,0.6562,0.5625,0.5714,0.4852,0.6565,0.1342


# Table III

In [68]:
pipelines = [
    'ARIMA',
    'LSTM-NDT',
    'LSTM-AE',
    'LSTM-VAE',
    'TadGAN'
]
non_aer_f1 = pd.concat([load_f1(pipeline) for pipeline in pipelines])


aer_pipelines = [
    'AER (PRED)',
    'AER (SUM)',
    'AER (REC)',
    'AER (MULT)'
]
aer_f1 = pd.concat([load_f1(pipeline) for pipeline in aer_pipelines])
aer_f1 = aer_f1.max(axis=0)
aer_f1.Mean = np.mean(aer_f1[:12])
aer_f1.StDev = np.std(aer_f1[:12])
aer_f1 = pd.DataFrame(aer_f1).T
aer_f1.index = ['AER']

pd.concat([non_aer_f1, aer_f1])

,MSL,SMAP,YAHOOA1,YAHOOA2,YAHOOA3,YAHOOA4,artificialWithAnomaly,realAdExchange,realAWSCloudwatch,realTraffic,realTweets,UCR,Mean,StDev
ARIMA,0.4421,0.3333,0.7332,0.8074,0.8176,0.6998,0.3529,0.7407,0.5176,0.5000,0.5672,0.1236,0.552900,0.206800
LSTM-NDT,0.5155,0.7075,0.7206,0.9801,0.7442,0.6379,0.4000,0.7407,0.5128,0.6667,0.5797,0.3908,0.633000,0.158900
LSTM-AE,0.4571,0.7259,0.6391,0.9610,0.5938,0.3678,0.4444,0.6429,0.6786,0.5926,0.5357,0.3292,0.580700,0.165200
LSTM-VAE,0.4324,0.6056,0.6171,0.9124,0.5977,0.3263,0.4444,0.7586,0.7018,0.5926,0.5357,0.3540,0.573200,0.162500
TadGAN,0.5843,0.6174,0.5325,0.8421,0.3907,0.2969,0.5714,0.7200,0.6774,0.5806,0.5882,0.1625,0.547000,0.177700
AER,0.6111,0.7969,0.7896,0.9728,0.8815,0.7281,0.8000,0.7333,0.7241,0.7368,0.6000,0.4868,0.738417,0.123916


# Figure 6